In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab

    # si usan colab, deben cambiar el token de esta url
    ! mkdir -p ../Data
    # los que usan colab deben modificar el token de esta url:
    ! wget -O ../Data/departamentos-en-venta-2016.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_07_Pandas_2/Data/departamentos-en-venta-2016.csv?token=AA4GFHO2DQ2XN4MJP3FHTLC6WSHME
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Pandas 2

## Introducción


En esta notebook vamos a trabajar con los conceptos de agregación a través de `groupby` y `pivot tables`. 

Analizaremos características de precio y superficie, por barrio y por comuna, de inmuebles en venta en la Ciudad de Buenos Aires en el año 2016.


## Dataset

El Gobierno de la Ciudad de Buenos Aires disponibiliza algunos datasets para acceso público en la url 
https://data.buenosaires.gob.ar/

Allí encontramos datos de propiedades inmuebles en venta, separados por año
https://data.buenosaires.gob.ar/dataset/departamentos-venta

En esta clase vamos a usar un subconjunto de los datos de "Departamentos en venta 2016" que descargamos desde aquí
https://data.buenosaires.gob.ar/dataset/departamentos-venta/archivo/juqdkmgo-7031-resource

En la carpeta Data de esta clase ya tenemos descargardo ese dataset, el nombre del archivo es **departamentos-en-venta-2016.csv**.

## Imports

In [ ]:
import pandas as pd
import numpy as np

## Ejercicio 1  - Importar 

Leamos los datos del archivo departamentos-en-venta-2016.csv

Veamos cuántos registros tiene y de qué tipos son los datos de cada columna. 

Veamos los primeros registros para verificar que los datos fueron importados correctamente.

In [ ]:
import chardet
def get_encoding_type(csv_path):
    rawdata = open(csv_path, 'rb').read()
    result = chardet.detect(rawdata)
    return result.get('encoding')

In [ ]:
data_location = "../Data/departamentos-en-venta-2016.csv"

In [ ]:
## [BORRAR_PRESENCIAL]
file_encoding = get_encoding_type(data_location)
file_encoding

In [ ]:
## [BORRAR_PRESENCIAL]
data = pd.read_csv(data_location, sep=";", encoding=file_encoding)
data.shape

In [ ]:
data.dtypes

In [ ]:
data.head()

## Ejercicio 2  - Promedio

¿Cuál es el promedio valor del precio por $m^2$ en dolares?

Para responder esta pregunta vamos a:

1) Crear un obeto Series que resulte de limpiar los valores del campo PRECIOTEXT. Tenemos que eliminar los símbolos "U\\$S" y "."

2) En la instancia de Series resultado del punto 1) reemplazamos los valores en pesos ('\\$') por nulos (para que nos ensucien los datos de promedio). 

3) Crear una columna nueva de tipo numérico en el DataFrame (PRECIOTEXT_CLEAN) y asignar los valores de resultado de 2)

3) Crear una columna de tipo numérico (PRECIOTEXTM) que tenga el precio del m2 en dolares calculado como el valor de PRECIOTEXT_CLEAN / M2

4) Calcular la media de los valores de PRECIOTEXTM


In [ ]:
import re

# reemplazo U$S por vacío
dolares_pattern = "U\$S\s*"
dolares_regex = re.compile(dolares_pattern)

# reemplazo el punto decimal por vacío
decimal_pattern = "\."
decimal_regex = re.compile(decimal_pattern)

sin_dolar = data.PRECIOTEXT.apply(lambda x: x if x is np.NaN else dolares_regex.sub("", x))

sin_dolar_sin_punto = sin_dolar.apply(lambda x: x if x is np.NaN else decimal_regex.sub("", x))

#type(sin_dolar_sin_punto)

In [ ]:
# armo una máscara con los registros que tienen el valor de PRECIOTEXT en $
pesos_pattern = "\$"
pesos_regex = re.compile(pesos_pattern)

matches_pesos = sin_dolar_sin_punto.apply(lambda x: x if x is np.NaN else pesos_regex.match(x))
pesos_mask = matches_pesos.notnull()
#pesos_mask.sum()

#veo que egfectivamente esos son los valores con $
print(sin_dolar_sin_punto[pesos_mask])

# asigno np.NaN a esos valores
sin_dolar_sin_punto[pesos_mask] = np.NaN


In [ ]:
sin_dolar_sin_punto_sin_pesos_num  = sin_dolar_sin_punto.astype(float)
data["PRECIOTEXT_CLEAN"] = sin_dolar_sin_punto_sin_pesos_num
#print(data.dtypes)
data["PRECIOTEXTM"] = data["PRECIOTEXT_CLEAN"] / data["M2"]
#data.head(3)
precio_promedio_m2 = data["PRECIOTEXTM"].mean()
precio_promedio_m2.round(2)

## Ejercicio 3  - Promedio por cuartil
¿Cuál es el precio promedio del metro cuadrado en dolares para cada cuartil de superficie (campo M2) de las viviendas en CABA?

Comenzamos calculando los cuartilos de superficie.

In [ ]:
q_superficie = pd.qcut(data.M2, 4)
q_superficie

In [ ]:
data.groupby(q_superficie)["PRECIOTEXTM"].mean().round(2)

## Ejercicio 3  - Promedio por barrio

¿Cuál es la media de precio por metro cuadrado en dolares para cada barrio de la CABA? 

Ordenemos los datos para indicar cuál es el barrio más caro.

Resolvamos el cálculo tanto con groupby como con pivot tables

In [ ]:
data.groupby('BARRIO')['PRECIOTEXTM'].mean().round(2).sort_values(ascending = False)

In [ ]:
data.pivot_table(index = 'BARRIO', aggfunc={'PRECIOTEXTM':'mean'}) \
                    . round(2) \
                    .sort_values(by='PRECIOTEXTM',ascending=False)

## Ejercicio 4  - Dispersión de precios por barrio

**4.a** ¿En qué barrio hay una mayor dispersión en el valor del metro cuadrado en dolares? Ordenar los valores para identificar el mayor.

Ayuda: Calculamos dispersión como el desvío estandar de un grupo dividido la media de ese grupo grupo

Vamos a calcular esto de dos maneras distintas:

1) Calculamos la instancia de Series que tiene la media por grupo. Calculamos la instancia de Series que tiene el desvío estandar por grupo. Las dividimos. Ordenamos

2) Calculamos los grupos y usamos una función lambda que calcule la media, desvío estandar y divida. Ordenamos.

**4.b** Cuál les parece que será más eficiente? Probemoslo con %timeit, para eso definamos dos funciones que encapsulen el código de 1) y 2).

**4.c** ¿Por qué Paternal está devolviendo NaN? ¿Y por qué Villa Soldati devuelve 0?

Nota: tengan en cuenta que `mean` y `std` sobre una instancia de `DataFrameGroupBy` excluye los valores nulos

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.mean.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.std.html





In [ ]:
def dispersion_1(data):
    barrio_group = data.groupby('BARRIO')
    medias_barrio = barrio_group['PRECIOTEXTM'].mean()
    desvios_barrio = barrio_group['PRECIOTEXTM'].std()
    dipersion = desvios_barrio / medias_barrio
    result = dipersion.sort_values(ascending=False) 
    return result

def dispersion_2(data):
    result = data.groupby('BARRIO')['PRECIOTEXTM'].apply(lambda x: x.std() / x.mean()).sort_values(ascending = False)
    return result
    

In [ ]:
dispersion_1(data)

In [ ]:
dispersion_2(data)

In [ ]:
%timeit dispersion_1(data)

In [ ]:
%timeit dispersion_2(data)

Vamos a ver cuánto da si usamos la lambda directamente, sin pasar por dispersion_2:

In [ ]:
%timeit data.groupby('BARRIO')['PRECIOTEXTM'].apply(lambda x: x.std() / x.mean()).sort_values(ascending = False)

Veamos por qué Paternal devuelve NaN y Villa Soldati que devuelve 0.

Para eso vamos a contar cuántos registros tenemos en cada barrio, en particular en Paternal.

In [ ]:
barrio_count = data.groupby('BARRIO')['BARRIO'].count()
print(barrio_count["PATERNAL"])

Hay un sólo registro en el barrio de Paternal, entonces el desvío está dividiendo por 0 y por eso devuelve NaN

In [ ]:
print(barrio_count["VILLA SOLDATI"])

villa_soldati_mask = data.BARRIO == "VILLA SOLDATI"
villa_soldati_data = data.loc[villa_soldati_mask, :]
villa_soldati_data["PRECIOTEXTM"]

Vemos que para Villa Soldati los dos registros tienen valores iguales, entonces el numerador en el cálculo del desvío estandar da 0, y por lo tanto también el desvío estandar y la dispersión resultan 0.


## Ejercicio 5  - Superficie por barrio

Calculemos la mediana de superficie por barrio para determinar en qué barrio los departamentos son más grandes.

Hagamos el cálculo con pivot_table y groupby


In [ ]:
data.pivot_table('M2', index='BARRIO', aggfunc=np.median).sort_values(by='M2', ascending=False)

In [ ]:
data.groupby('BARRIO')['M2'].median().sort_values(ascending=False)

## Ejercicio 6  - Tabla resumen

Generemos un `DataFrame` que agregue la información del precio por M2 en dolares (`PRECIOTEXTM`), ambientes (`AMBIENTES`) a nivel de `COMUNA` y barrio (`BARRIO`). Proporcione información tanto de la tendencia central como de la dispersión de ambas distribuciones.

Ayuda: usar pivot_table

In [ ]:
result = data.pivot_table(['PRECIOTEXTM','AMBIENTES'],index=['COMUNA','BARRIO']
               , aggfunc={'PRECIOTEXTM':[np.mean,np.std,len],
                          'AMBIENTES': [np.mean,np.std]})
print(type(result))
result